In [1]:
from mealpy import Problem, FloatVar, BBO, IntegerVar, GA, StringVar
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from mealpy import GA, PSO, Problem, FloatVar, IntegerVar
import numpy as np

SEED = 42

2024-10-11 12:00:17.890183: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-11 12:00:17.893647: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-11 12:00:17.906284: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-11 12:00:17.925574: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-11 12:00:17.930328: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-11 12:00:17.945279: I tensorflow/core/platform/cpu_feature_gu

In [2]:
data = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=SEED)

In [4]:
class LogisticRegression():
    def __init__(self, input: int, optimizer: str, learning_rate: float):
        self.input = input
        self.optimizer_name = optimizer
        self.learning_rate = learning_rate
        self.sequential = Sequential([
            Input(shape=(input,)),
            Dense(1, activation='linear'),
        ])

    def fit(self, X, y, epochs=50):
        self.sequential.compile(
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=getattr(tf.keras.optimizers, self.optimizer_name)(learning_rate=self.learning_rate))
        history = self.sequential.fit(X, y, epochs=epochs, verbose=0)
        last_loss = history.history["loss"][-1]
        return last_loss



In [7]:
class HyperParamProblem(Problem):
    def __init__(self, bounds, minmax="min", data=None, **kwargs):
        self.data = data

        super().__init__(bounds, minmax, **kwargs)

    def train_evaluate_model(self, optimizer_name, learning_rate):
        input_dim = self.data["X_train"].shape[1]
        model = LogisticRegression(input=input_dim, optimizer=optimizer_name, learning_rate=learning_rate)
        loss_train = model.fit(self.data["X_train"], self.data["y_train"], epochs=self.data["epoch"])
        loss_test = model.sequential.evaluate(self.data["X_test"], self.data["y_test"], verbose=0)

        return [loss_train, loss_test]

    def obj_func(self, solution):
        solution = self.decode_solution(solution)
        optimizer_name, learning_rate = solution["optimizer"], solution["learning_rate"]
        train_test_loss = self.train_evaluate_model(optimizer_name, learning_rate)
        return train_test_loss

In [9]:
data = {
    "X_train": X_train,
    "y_train": y_train,
    "X_test": X_test,
    "y_test": y_test,
    "epoch": 100
}
optimizers = [ "SGD", "Adam", "RMSprop", "Adagrad", "Adadelta", "Adamax"]
bounds = [
    StringVar(valid_sets=optimizers, name="optimizer"),
    FloatVar(lb=0, ub=0.1, name="learning_rate"),
]

In [10]:
hyperparam_problem = HyperParamProblem(bounds=bounds, minmax="min", data=data, name="Hyperparam_problem", obj_weights=(0.5, 0.5))

In [11]:
ga_model = GA.BaseGA(epoch=5, pop_size=50)
ga_best = ga_model.solve(hyperparam_problem, mode='thread')

2024/10/11 12:02:20 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: Solving 2-objective optimization problem with weights: [0.5 0.5].
2024/10/11 12:02:20 PM, WARNING, mealpy.evolutionary_based.GA.BaseGA: The parallel mode: thread is selected. But n_workers is not set. The default n_workers = 4 is used.


2024/10/11 12:06:29 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: Hyperparam_problem, Epoch: 1, Current best: 0.10121588036417961, Global best: 0.10121588036417961, Runtime: 125.07289 seconds
2024/10/11 12:08:31 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: Hyperparam_problem, Epoch: 2, Current best: 0.10421675443649292, Global best: 0.10121588036417961, Runtime: 122.62916 seconds
2024/10/11 12:10:37 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: Hyperparam_problem, Epoch: 3, Current best: 0.08724456839263439, Global best: 0.08724456839263439, Runtime: 126.02542 seconds


2024/10/11 12:12:47 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: Hyperparam_problem, Epoch: 4, Current best: 0.09867200814187527, Global best: 0.08724456839263439, Runtime: 130.17507 seconds
2024/10/11 12:14:51 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: Hyperparam_problem, Epoch: 5, Current best: 0.09203901328146458, Global best: 0.08724456839263439, Runtime: 123.57800 seconds


In [12]:
print(f"Best agent: {ga_best}")
print(f"Best solution: {ga_best.solution}")
print(f"Best accuracy: {ga_best.target.fitness}")
print(f"Best parameters: {ga_model.problem.decode_solution(ga_best.solution)}")

Best agent: id: 566, target: Objectives: [0.11708042 0.05740872], Fitness: 0.08724456839263439, solution: [3.         0.02384694]
Best solution: [3.         0.02384694]
Best accuracy: 0.08724456839263439
Best parameters: {'optimizer': 'Adamax', 'learning_rate': 0.023846939637753486}


In [13]:
bbo_model = BBO.OriginalBBO(epoch=5, pop_size=20, seed=SEED)
bbo_best = bbo_model.solve(hyperparam_problem, mode="thread")

2024/10/11 12:14:51 PM, INFO, mealpy.bio_based.BBO.OriginalBBO: Solving 2-objective optimization problem with weights: [0.5 0.5].
2024/10/11 12:14:51 PM, WARNING, mealpy.bio_based.BBO.OriginalBBO: The parallel mode: thread is selected. But n_workers is not set. The default n_workers = 4 is used.


2024/10/11 12:16:27 PM, INFO, mealpy.bio_based.BBO.OriginalBBO: >>>Problem: Hyperparam_problem, Epoch: 1, Current best: 0.12737616151571274, Global best: 0.12737616151571274, Runtime: 48.28135 seconds
2024/10/11 12:17:14 PM, INFO, mealpy.bio_based.BBO.OriginalBBO: >>>Problem: Hyperparam_problem, Epoch: 2, Current best: 0.10119216702878475, Global best: 0.10119216702878475, Runtime: 47.57103 seconds
2024/10/11 12:18:02 PM, INFO, mealpy.bio_based.BBO.OriginalBBO: >>>Problem: Hyperparam_problem, Epoch: 3, Current best: 0.10119216702878475, Global best: 0.10119216702878475, Runtime: 47.93851 seconds
2024/10/11 12:18:52 PM, INFO, mealpy.bio_based.BBO.OriginalBBO: >>>Problem: Hyperparam_problem, Epoch: 4, Current best: 0.10116437450051308, Global best: 0.10116437450051308, Runtime: 49.25821 seconds
2024/10/11 12:19:39 PM, INFO, mealpy.bio_based.BBO.OriginalBBO: >>>Problem: Hyperparam_problem, Epoch: 5, Current best: 0.10116437450051308, Global best: 0.10116437450051308, Runtime: 47.64880 sec

In [14]:
print(f"Best agent: {bbo_best}")
print(f"Best solution: {bbo_best.solution}")
print(f"Best accuracy: {bbo_best.target.fitness}")
print(f"Best parameters: {bbo_model.problem.decode_solution(bbo_best.solution)}")

Best agent: id: 728, target: Objectives: [0.14003725 0.0622915 ], Fitness: 0.10116437450051308, solution: [3.         0.01025408]
Best solution: [3.         0.01025408]
Best accuracy: 0.10116437450051308
Best parameters: {'optimizer': 'Adamax', 'learning_rate': 0.01025407665488003}


In [15]:
best_solution = ga_model.problem.decode_solution(ga_best.solution) if bbo_best.target.fitness < ga_best.target.fitness else bbo_model.problem.decode_solution(bbo_best.solution)

In [16]:
best_solution

{'optimizer': 'Adamax', 'learning_rate': 0.01025407665488003}